In [21]:
import pandas as pd

In [22]:
df = pd.read_csv("spotify_millsongdata.csv")

In [23]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [24]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [25]:
df.shape

(57650, 4)

In [26]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [27]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [28]:
df.head(10)

,artist,song,text
0,Glen Campbell,He,He can turn the ties and calm the angry sea \...
1,Cheap Trick,Through The Night,You don't have to hide \r\nAll the things you...
2,Avril Lavigne,Destiny,Destiny \r\n \r\nSearching for my destiny \...
3,The Jam,HEY MISTER,Because it's real you think it's a joke \r\nB...
4,Fall Out Boy,7 Minutes In Heaven,I'm sleeping my way out of this one \r\nWith ...
5,Amy Grant,The Power,A king is crying from his throne \r\nHe had a...
6,Indiana Bible College,Set Me Free,"[Chorus:] \r\nI don't even know, where my lif..."
7,Jennifer Lopez,First Love,See I know a little bit something good \r\nAl...
8,Tim McGraw,You Don't Love Me Anymore,She walks over to him and she says \r\nDo you...
9,Selah,Sweet Jesus,"Sweet Jesus, my Savior \r\nYou are my faithfu..."


In [29]:
df['text'][0]

"He can turn the ties and calm the angry sea  \r\nHe alone besides who writes a symphony  \r\nHe lights every star that makes a darkness bright  \r\nHe keeps watch all through each long and lonely night  \r\nHe still finds the time to hear a child's first prayer  \r\nSaint or sinner call and always find him there  \r\nThough it makes him sad to see the way we live he'll always say I forgive  \r\n(He can grab a wish or make a dream come true)  \r\nHe can paint the clouds and turn the gray to blue)  \r\nHe alone knows where to find a rainbow's end  \r\nHe alone can see what lies beyond the bend  \r\nHe can touch a tree and turn the leaves to gold  \r\nHe knows every lie that you and I have told  \r\nThough it makes him sad to see the way we live he'll always say I forgive\r\n\r\n"

In [30]:
df = df.sample(5000)

In [31]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [32]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [33]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [34]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [37]:
similarity[0]

array([1.        , 0.09165176, 0.07103868, ..., 0.02282405, 0.02854422,
       0.02661597])

In [38]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [39]:
def recommendation(song_df):
    if 'song' not in df.columns:
        return "Error: 'song' column not found in the DataFrame."

    try:
        idx = df[df['song'] == song_df].index[0]
    except IndexError:
        return f"Error: '{song_df}' not found in the 'song' column."

    distances = sorted(
        list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    songs = []

    # Continue with the rest of your code to recommend songs based on similarities


In [40]:
recommendation('Crying Over You')

"Error: 'Crying Over You' not found in the 'song' column."

In [41]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))